In [4]:
import pandas as pd
import xgboost as xgb
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve

In [5]:
data = pd.read_csv("source/train.csv")

In [6]:
data

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.50,Premium,F,VS2,61.5,58.0,7.32,7.34,4.51,9.588
1,1,2.01,Very Good,E,SI2,60.6,59.0,8.11,8.25,4.96,9.748
2,2,0.50,Ideal,E,SI1,61.6,57.0,5.13,5.09,3.15,7.255
3,3,0.25,Very Good,F,VVS2,61.6,57.0,4.05,4.08,2.50,6.450
4,4,0.52,Ideal,G,VS2,62.0,55.0,5.16,5.19,3.21,7.721
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,1.04,Very Good,I,SI2,59.6,57.0,6.60,6.62,3.94,8.190
40451,40451,0.51,Very Good,E,SI1,63.3,59.0,5.09,5.05,3.21,7.246
40452,40452,1.51,Ideal,H,VS2,62.6,57.0,7.37,7.33,4.60,9.277
40453,40453,2.02,Premium,H,VS2,61.3,60.0,8.16,8.11,4.99,9.680


#### En vez de hacer dummies ,  vamos a ver si estas variables float tienen un orden de importancia comparandolas con el precio.

In [7]:
data["price/carat"]= data["price"] / data["carat"]

In [8]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price,price/carat
0,0,1.50,Premium,F,VS2,61.5,58.0,7.32,7.34,4.51,9.588,6.392000
1,1,2.01,Very Good,E,SI2,60.6,59.0,8.11,8.25,4.96,9.748,4.849751
2,2,0.50,Ideal,E,SI1,61.6,57.0,5.13,5.09,3.15,7.255,14.510000
3,3,0.25,Very Good,F,VVS2,61.6,57.0,4.05,4.08,2.50,6.450,25.800000
4,4,0.52,Ideal,G,VS2,62.0,55.0,5.16,5.19,3.21,7.721,14.848077


In [9]:
print(data.groupby('cut')['price/carat'].mean().sort_values())
print(data.groupby('color')['price/carat'].mean().sort_values())
print(data.groupby('clarity')['price/carat'].mean().sort_values())

cut
Fair          9.284466
Good         11.663682
Premium      11.807049
Very Good    12.446719
Ideal        13.893712
Name: price/carat, dtype: float64
color
J     9.017189
I    10.422280
H    11.510122
G    12.992913
F    13.141893
D    14.133644
E    14.300001
Name: price/carat, dtype: float64
clarity
I1       7.466673
SI2      9.212076
SI1     11.440938
VS2     13.008773
VS1     13.512145
VVS2    15.815193
VVS1    17.459082
IF      17.667809
Name: price/carat, dtype: float64


In [10]:
data['cut']=data['cut'].replace({'Ideal':1,'Good':2,'Very Good':3,'Fair':4,'Premium':5})
data['color']=data['color'].replace({'E':1,'D':2,'F':3,'G':4,'H':5,'I':6,'J':7})
data['clarity']=data['clarity'].replace({'VVS1':1,'IF':2,'VVS2':3,'VS1':4,'I1':5,'VS2':6,'SI1':7,'SI2':8})

In [13]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.50,5,3,6,61.5,58.0,7.32,7.34,4.51,9.588
1,1,2.01,3,1,8,60.6,59.0,8.11,8.25,4.96,9.748
2,2,0.50,1,1,7,61.6,57.0,5.13,5.09,3.15,7.255
3,3,0.25,3,3,3,61.6,57.0,4.05,4.08,2.50,6.450
4,4,0.52,1,4,6,62.0,55.0,5.16,5.19,3.21,7.721


In [11]:
data = data.drop(["price/carat"], axis=1)

In [12]:
data.corr()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
id,1.000000,-0.006196,-0.010850,0.002469,-0.002531,0.001098,-0.008829,-0.007112,-0.006420,-0.007766,-0.006926
carat,-0.006196,1.000000,0.163227,0.289227,0.320217,0.025089,0.182590,0.974516,0.950563,0.947930,0.920878
cut,-0.010850,0.163227,1.000000,0.040410,0.164484,-0.087940,0.499487,0.167382,0.150368,0.145017,0.123236
color,0.002469,0.289227,0.040410,1.000000,-0.023420,0.044665,0.024083,0.269491,0.263042,0.265130,0.159381
clarity,-0.002531,0.320217,0.164484,-0.023420,1.000000,0.044650,0.149458,0.343427,0.331780,0.335356,0.206736
depth,0.001098,0.025089,-0.087940,0.044665,0.044650,1.000000,-0.300152,-0.028389,-0.032561,0.091421,-0.000924
table,-0.008829,0.182590,0.499487,0.024083,0.149458,-0.300152,1.000000,0.196031,0.184322,0.150142,0.158128
x,-0.007112,0.974516,0.167382,0.269491,0.343427,-0.028389,0.196031,1.000000,0.973617,0.965597,0.957653
y,-0.006420,0.950563,0.150368,0.263042,0.331780,-0.032561,0.184322,0.973617,1.000000,0.945958,0.934700
z,-0.007766,0.947930,0.145017,0.265130,0.335356,0.091421,0.150142,0.965597,0.945958,1.000000,0.930157


In [14]:
data.describe()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,20227.000000,0.796440,2.648177,3.545248,5.516821,61.747166,57.455860,5.727615,5.730717,3.536446,7.782551
std,11678.496907,0.473467,1.619787,1.767992,2.086539,1.433123,2.233779,1.121224,1.143828,0.708732,1.014333
min,0.000000,0.200000,1.000000,1.000000,1.000000,43.000000,43.000000,0.000000,0.000000,0.000000,5.787000
25%,10113.500000,0.400000,1.000000,2.000000,4.000000,61.000000,56.000000,4.710000,4.720000,2.910000,6.853000
50%,20227.000000,0.700000,3.000000,4.000000,6.000000,61.800000,57.000000,5.690000,5.700000,3.520000,7.779000
75%,30340.500000,1.040000,5.000000,5.000000,7.000000,62.500000,59.000000,6.540000,6.540000,4.030000,8.576000
max,40454.000000,5.010000,5.000000,7.000000,8.000000,79.000000,95.000000,10.740000,58.900000,31.800000,9.843000


### Train - test split 

In [14]:
from sklearn.model_selection import train_test_split

In [27]:
from sklearn.metrics import mean_squared_error

In [15]:
data_train, data_test = train_test_split (data, test_size = 0.3, random_state= 100)

In [16]:
data_train_mat = xgb.DMatrix(data_train.drop("price", 1), label=data_train["price"])
data_test_mat = xgb.DMatrix(data_test.drop("price", 1), label=data_test["price"])

In [17]:
data_train_mat

In [65]:
parametros = {"booster":"gbtree", "max_depth": 9, "eta": 0.3, "objective": "reg:squarederror", "nthread":2}
rondas = 10

In [66]:
modelo = xgb.train(parametros, data_test_mat, rondas)

In [67]:
modelo

In [68]:
prediccion = modelo.predict(data_test_mat)

In [69]:
prediccion

array([6.5321903, 9.15392  , 6.6181808, ..., 6.622279 , 7.4158463,
       8.077966 ], dtype=float32)

In [70]:
mean_squared_error(data_test["price"], prediccion)

0.05668961585835056

In [ ]:
## Diamonds TEST

In [54]:
data_test = pd.read_csv("source/test.csv")

In [55]:
data_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.51,Very Good,E,SI2,58.3,61.9,5.19,5.20,3.04
1,1,1.02,Very Good,F,SI2,63.0,58.0,6.37,6.43,4.03
2,2,0.59,Ideal,D,VS2,61.9,56.0,5.39,5.34,3.32
3,3,0.90,Very Good,D,VS2,62.3,56.0,6.14,6.18,3.84
4,4,2.01,Premium,J,VS1,60.2,61.0,8.23,8.16,4.93


In [56]:
data_test['cut']=data_test['cut'].replace({'Ideal':1,'Good':2,'Very Good':3,'Fair':4,'Premium':5})
data_test['color']=data_test['color'].replace({'E':1,'D':2,'F':3,'G':4,'H':5,'I':6,'J':7})
data_test['clarity']=data_test['clarity'].replace({'VVS1':1,'IF':2,'VVS2':3,'VS1':4,'I1':5,'VS2':6,'SI1':7,'SI2':8})

In [60]:
data_test = data_test.drop(columns = ["id"])

In [61]:
data_test.describe()

,carat,cut,color,clarity,depth,table,x,y,z
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,0.802440,2.660141,3.516945,5.501965,61.756122,57.461157,5.741783,5.745953,3.545598
std,0.475629,1.615232,1.767303,2.108788,1.431146,2.236702,1.123344,1.137006,0.696500
min,0.200000,1.000000,1.000000,1.000000,52.700000,49.000000,0.000000,0.000000,0.000000
25%,0.400000,1.000000,2.000000,4.000000,61.000000,56.000000,4.720000,4.730000,2.920000
50%,0.710000,3.000000,4.000000,6.000000,61.800000,57.000000,5.710000,5.720000,3.530000
75%,1.040000,5.000000,5.000000,7.000000,62.500000,59.000000,6.540000,6.540000,4.040000
max,4.000000,5.000000,7.000000,8.000000,79.000000,79.000000,10.010000,31.800000,6.380000


In [ ]:
data_test.head()

In [62]:
### 

test_predictions = rf.predict(data_test)

In [64]:
data_test.reset_index()

,index,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.51,3,1,8,58.3,61.9,5.19,5.20,3.04
1,1,1.02,3,3,8,63.0,58.0,6.37,6.43,4.03
2,2,0.59,1,2,6,61.9,56.0,5.39,5.34,3.32
3,3,0.90,3,2,6,62.3,56.0,6.14,6.18,3.84
4,4,2.01,5,7,4,60.2,61.0,8.23,8.16,4.93
...,...,...,...,...,...,...,...,...,...,...
13480,13480,1.05,5,5,7,61.9,58.0,6.50,6.45,4.01
13481,13481,0.32,1,3,6,62.1,57.0,4.35,4.38,2.71
13482,13482,0.41,5,7,4,61.3,59.0,4.78,4.74,2.92
13483,13483,0.30,5,4,3,62.2,59.0,4.31,4.28,2.67


In [68]:
data_test2 = pd.read_csv("source/test.csv")
data_test2.columns

Index(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z'],
      dtype='object')

In [70]:
columnas = ['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z']

In [71]:
data_test["id"]= data_test2["id"]

In [73]:
test_ID = data_test["id"]

submission_2 = pd.DataFrame({
    "id": test_ID,
    "Price": test_predictions
})

In [74]:
submission_2

,id,Price
0,0,7.089158
1,1,8.343485
2,2,7.662998
3,3,8.474553
4,4,9.545008
...,...,...
13480,13480,8.473776
13481,13481,6.455610
13482,13482,6.689342
13483,13483,6.762749


In [75]:
submission_2.to_csv('submission_2.csv', index=False, header = True)